In [ ]:
# cd to the root directory
!python setup.py install
!qiime dev refresh-cache

In [28]:
import pandas as pd
import numpy as np
import qiime2 as q2
import zarr
import os


import matplotlib.pyplot as plt
from scipy import stats

from q2_gglasso.utils import PCA, remove_biom_header

### Import data

In [ ]:
!qiime tools import \
   --type EMPPairedEndSequences \
   --input-path data/atacama-emp-paired-end-sequences \
   --output-path data/atacama-emp-paired-end-sequences.qza

### Demultiplexing
"Demultiplexing" refers to the step in processing where you'd use the barcode information in order to know which sequences came from which samples after they had all been sequenced together. In this data set, the barcode reads are the reverse complement of those included in the sample metadata file, so we additionally include the --p-rev-comp-mapping-barcodes parameter.

In [ ]:
!qiime demux emp-paired \
  --m-barcodes-file data/atacama-sample-metadata.tsv \
  --m-barcodes-column barcode-sequence \
  --p-rev-comp-mapping-barcodes \
  --i-seqs data/atacama-emp-paired-end-sequences.qza \
  --o-per-sample-sequences data/atacama-demux-full.qza \
  --o-error-correction-details data/atacama-demux-details.qza

In [ ]:
!qiime demux summarize \
  --i-data data/atacama-demux-full.qza \
  --o-visualization data/atacama-demux-full.qzv

In [ ]:
!qiime tools export \
  --input-path data/atacama-demux-full.qzv \
  --output-path data/atacama-demux-full/

### Filtering short sequences

In [ ]:
!qiime demux filter-samples \
  --i-demux data/atacama-demux-full.qza \
  --m-metadata-file data/atacama-demux-full/per-sample-fastq-counts.tsv \
  --p-where 'CAST([forward sequence count] AS INT) > 100' \
  --o-filtered-demux data/atacama-demux.qza

### Denoising with DADA2
use recommended trimming pararms

In [ ]:
!qiime dada2 denoise-paired \
  --i-demultiplexed-seqs data/atacama-demux.qza \
  --p-trim-left-f 13 \
  --p-trim-left-r 13 \
  --p-trunc-len-f 150 \
  --p-trunc-len-r 150 \
  --o-table data/atacama-table.qza \
  --o-representative-sequences data/atacama-rep-seqs.qza \
  --o-denoising-stats data/atacama-denoising-stats.qza

### Summary of the feature table

In [ ]:
!qiime feature-table summarize \
  --i-table data/atacama-table.qza \
  --o-visualization data/atacama-table.qzv \
  --m-sample-metadata-file data/atacama-sample-metadata.tsv

Check features and representative sequences

In [ ]:
!qiime feature-table tabulate-seqs \
  --i-data data/atacama-rep-seqs.qza \
  --o-visualization data/atacama-rep-seqs.qzv

### Check denoising stats

In [ ]:
!qiime metadata tabulate \
  --m-input-file data/atacama-denoising-stats.qza \
  --o-visualization data/atacama-denoising-stats.qzv

### Select top100 taxa

In [ ]:
!qiime feature-table filter-features \
    --i-table data/atacama-table.qza \
    --o-filtered-table data/atacama-table.qza \
    --p-min-frequency 100

### Replace zeros with ones

In [ ]:
!qiime composition add-pseudocount \
                    --i-table data/atacama-table.qza \
                    --p-pseudocount 1 \
                    --o-composition-table data/atacama-table_composition.qza

### Transform compositional data with CLR

In [ ]:
!qiime gglasso transform-features \
     --p-transformation clr \
     --i-table data/atacama-table_composition.qza \
     --o-transformed-table data/atacama-table_clr.qza

In [ ]:
!qiime gglasso calculate-covariance \
     --p-method scaled \
     --i-table data/atacama-table_clr.qza \
     --o-covariance-matrix data/atacama-table_corr.qza

In [ ]:
!qiime gglasso solve-problem \
     --p-lambda1 0.07 \
     --p-latent True \
     --p-mu1 2.2 1.8 1.6 1.5 1.4 1.2 1.15 1.05 1 0.95 \
     --i-covariance-matrix data/atacama-table_corr.qza \
     --o-solution data/atacama-solution_low.qza \
     --verbose

### Export

In [ ]:
!qiime tools export \
  --input-path data/atacama-table_clr.qza \
  --output-path data/atacama-table_clr

In [ ]:
!qiime tools export \
  --input-path data/atacama-table_corr.qza \
  --output-path data/atacama-table_corr

In [ ]:
!biom convert -i data/atacama-table_clr/feature-table.biom -o data/atacama-table_clr/clr_feature-table.tsv --to-tsv

In [ ]:
remove_biom_header(file_path="data/atacama-table_clr/clr_feature-table.tsv")

### Plotting low rank solution with metadata

In [ ]:
sol = zarr.load("data/atacama_low/problem.zip")
L = sol['solution/lowrank_']

df = pd.read_csv(str("data/atacama-table_clr_small/small_clr_feature-table.tsv"), index_col=0, sep='\t')

In [ ]:
mapping = pd.read_csv("data/atacama-sample-metadata.tsv", sep='\t', index_col=0)
mapping = mapping.reindex(df.index)
mapping.vegetation = mapping.vegetation.map(dict(yes=1, no=0))

cols = mapping.columns.drop(['barcode-sequence', 'extract-group-no',
                             'transect-name', 'site-name', 'vegetation'])

mapping[cols] = mapping[cols].apply(pd.to_numeric, errors='coerce')

In [ ]:
for col in mapping[cols]:
    if col != 'depth':
        path = "example/atacama/plots/" + col
        if not os.path.exists(path):
            os.makedirs(path)

        plot_df = df.join(mapping[col])
        plot_df = plot_df.dropna()
        print(col, ":", plot_df.shape)

        proj, loadings, eigv = PCA(plot_df.iloc[:, :-1], L, inverse=True)
        r = np.linalg.matrix_rank(L)

        plot_df = plot_df.join(mapping['depth'])
        depth = plot_df['depth']

        for i in range(0, r):
            fig, ax = plt.subplots(1, 1)
            im = ax.scatter(proj[:, i], plot_df[col], c=depth, cmap=plt.cm.Blues, vmin=0)
            cbar = fig.colorbar(im)
            cbar.set_label("Sampling depth")
            ax.set_xlabel("PCA component {0} with eigenvalue {1}".format(i + 1, eigv[i]))
            ax.set_ylabel("{0}".format(col))
            plt.savefig('example/atacama/plots/{0}/{1}_r{2}.png'.format(col, col, i + 1))

            print("Spearman correlation between {0} and {1} component: {2}, p-value: {3}".format(col, i+1,
                stats.spearmanr(plot_df[col], proj[:, i])[0],
                stats.spearmanr(plot_df[col], proj[:, i])[1]))